In [908]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [909]:
import os
import pandas
import json
import re #remove_tags function
os.chdir('C:\\python 3.8\\app0')
os.getcwd()

'C:\\python 3.8\\app0'

In [910]:
#+ function
##  remove tag
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [911]:
filedir = 'data\\one_branch.csv'
filedir_all = os.getcwd() + '\\' + filedir
print(filedir_all)

data = pandas.read_csv(filedir_all)

#needed zip upload 124 + convert to pandas data frame
data124 = data[data.course_id == 124]
data124 = pandas.DataFrame(data124)


C:\python 3.8\app0\data\one_branch.csv


In [912]:
#convert ['q_update_ts'] to time#
#################################
data124['q_update_ts'] = pandas.to_datetime(data124['q_update_ts'])

In [913]:
#pure text

data124['text'] = data124['prompt'].apply(lambda x: 
                     remove_tags(json.loads(x)['definition']['value']))
#delete quitation mark
#data124['text'] =  data124['text'].str.replace('"', '') #delete quotes


In [914]:
#print(data124['update_ts'].unique())
print('max ' + data124['update_ts'].max())
print('min ' + data124['update_ts'].min())


max 2019-02-27 14:13:00.295
min 2017-07-30 09:25:04.691


In [915]:
## create group for future aggregate

#position id
grouped = data124.groupby(['m_ord', 'l_ord', 'i_ord', 'subi_ord', 'aq_ord'])

#position - aq_ord + internal_id ##'i_ord', 'subi_ord',
grouped_ext_id = data124.groupby(['m_ord', 'l_ord',  'internal_id'])

In [916]:
#tab0 OR статистика posit id
tab0 = grouped.agg(question_ext_id = ('question_ext_id', 'nunique'),
                   text = ('text', 'nunique'),
                   internal_id = ('internal_id', 'nunique'),
                   cnt_pos = ('cnt', 'sum')
                  ).reset_index()

tab0.to_csv('data\\migrated internal_id\\tab0.csv', sep = ';', encoding='utf-8', index=False)

In [917]:
#sum of cnt
#maximum time of change 'update_ts'
res = grouped_ext_id.agg(question_ext_id = ('question_ext_id', 'nunique'),
                         UNIQUEtext = ('text', 'nunique'),
                         cnt = ('cnt', 'sum'),
                         update_ts = ('update_ts', 'max')                         
                        ).reset_index()

#stat!
print('res.shape -->')
print(res.shape)
#print('res.columns -->')
#print(res.columns)
print(res[res.m_ord == 4])

res.shape -->
(76, 7)
    m_ord  l_ord               internal_id  question_ext_id  UNIQUEtext   cnt  \
50      4      0  -wrc6Xn5EeeuOwpUSrH9EA@0                3           2  1466   
51      4      0  2FH6wnn8EeeuOwpUSrH9EA@0                3           1  1466   
52      4      0  CEyubnn5EeeuOwpUSrH9EA@0                3           2  1466   
53      4      0  FRTbmXn7EeeYfg4705Ecjg@0                5           3  1466   
54      4      0  FxhwbXn3EeeYfg4705Ecjg@0                5           1  1466   
55      4      0  O3xw93n8EeeLFhJpxYB-cA@0                4           2  1466   
56      4      0  SITYY3n6EeeLFhJpxYB-cA@0                3           3  1466   
57      4      0  dXlxRXn5EeeuOwpUSrH9EA@0                3           2  1466   
58      4      0  hbI7XXn4EeeYfg4705Ecjg@0                2           2   263   
59      4      2  AcUTL540Eee3phJkV4lW2g@0                1           1   574   
60      4      2  DbXB0541Eee3phJkV4lW2g@0                1           1   574   
61    

In [918]:
#create posit for res ##'i_ord', 'subi_ord', 
column_for_posit = ['m_ord', 'l_ord', 'internal_id', 'update_ts']
#
res['posit'] = res[column_for_posit].astype(str).apply('|'.join,1)
#print('res.columns -->')
#res.columns

In [919]:
#create posit for original -не совсем position
data124['posit'] = data124[column_for_posit].astype(str).apply('|'.join,1)

#data124.head
data124_pos_text = data124[['posit', 'text']]
print(data124_pos_text.shape)

#drop_duplicates
data124_pos_text = data124_pos_text.drop_duplicates(subset = 'posit') 
print(data124_pos_text.shape)

#save
##data124_pos_text.to_csv('data\\migrated internal_id\\origdata124_posit_abbr.csv', encoding='utf-8')

(319, 2)
(214, 2)


In [920]:
#merge two data frames
print('res.shape -->')
print(res.shape)

result = pandas.merge(res, data124_pos_text, how='left', on='posit', indicator=False)

print('result.shape -->')
print(result.shape)
print('result.columns -->')
print(result.columns)

#result

res.shape -->
(76, 8)
result.shape -->
(76, 9)
result.columns -->
Index(['m_ord', 'l_ord', 'internal_id', 'question_ext_id', 'UNIQUEtext', 'cnt',
       'update_ts', 'posit', 'text'],
      dtype='object')


In [921]:
#ord data by l + i + subi + aq_ord + cnt ##- 'i_ord', 'subi_ord',
ord_columns = ['m_ord', 'l_ord',  'cnt']
result = result.sort_values(by=ord_columns)

#look m_ord == 4
##mydata = result[result.m_ord == 4]
##mydata.to_csv('data\\migrated internal_id\\data_m_ord4.csv', sep = ';', encoding='utf-8', index=False)

In [922]:
#print(result.dtypes) ##, 'i_ord', 'subi_ord'
column_for_posit = ['m_ord', 'l_ord'] #for ranking
result['pos_for_rank'] = result[column_for_posit].astype(str).apply(''.join,1)

# convert column 'pos_for_rank' to numeric
result['pos_for_rank_num'] = pandas.to_numeric(result['pos_for_rank'])


result['x_pos'] = result.groupby('m_ord')['pos_for_rank_num'].rank(method='first').astype(int)
result['y_pos'] = result['m_ord'] + 1

needed_col = ['m_ord', 'l_ord', 'y_pos', 'x_pos',  'internal_id', 'question_ext_id', 'UNIQUEtext', 'update_ts', 'cnt',  'text']
result = result[needed_col]

result.to_csv('data\\migrated internal_id\\tab1.csv', sep = ';', encoding='utf-8', index=False)



In [923]:
####grouped_ext_id--2#### indexes!
## res = grouped_ext_id.agg(cnt = ('cnt', 'sum'),
##                          update_ts = ('update_ts', 'max'))
#pandas.DataFrame(res)

In [924]:
####grouped_ext_id####

#res_ext_id = grouped_ext_id['cnt'].sum()
#res_ext_id.to_csv('data\\agg_ext_id_sum.csv', encoding='utf-8')

#last changed date time in internal_id             
#res = grouped_ext_id['update_ts'].agg('max')

#res.to_csv('data\\last_date_time__internal_id.csv', encoding='utf-8')

In [925]:
#result.groupby(['m_ord'])


#result['pos_id'] = result[['m_ord', 'l_ord', 'i_ord', 'subi_ord']].astype(str).apply('|'.join,1)

#stat_result = result.groupby(['m_ord']).agg(
#    posit_id = ( 'pos_id', 'nunique'),
#    internal_id = ('internal_id', 'nunique')
#    ).reset_index()

#print(result[result.m_ord == 4])
#stat_result.to_csv('data\\statistics-2_tab1.csv', sep = ';', encoding='utf-8', index=False)
